In [1]:
import os
import math
import numpy as np

def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences
#x = read_fasta_file('test.fa')
#print(x['genomeA'])

#log-function
def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)

#define class
class hmm:
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = init_probs
        self.trans_probs = trans_probs
        self.emission_probs = emission_probs

#helper functions
def translate_observations_to_indices(obs):
    mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    return [mapping[symbol.lower()] for symbol in obs]

def translate_indices_to_observations(indices):
    mapping = ['a', 'c', 'g', 't']
    return ''.join(mapping[idx] for idx in indices)

def translate_path_to_indices(path):
    return list(map(lambda x: int(x), path))

def translate_indices_to_path(indices):
    mapping = ['C', 'C', 'C', 'N', 'R', 'R', 'R']
    return ''.join([mapping[i] for i in indices])


def translate_z_to_indices(path):
    out = []
    forwardcoding = 2
    backwardcoding = 0
    for i in path:
        if i == 'N':
            out.append(3)
        if i == 'C':
            out.append(forwardcoding)
            forwardcoding=(forwardcoding - 1) % 3
        if i == 'R':
            out.append(backwardcoding + 4)
            backwardcoding=(backwardcoding + 1) % 3
    return  out

def count_transitions_and_emissions(K, D, x, z):
    """
    Returns a KxK matrix and a KxD matrix containing counts cf. above
    """
    KK=np.ones((K,K))
    KD=np.ones((K,D))
    
    for X,Z in zip(x,z):
        KD[Z,X]+=1
    KD=[np.array(L)/sum(L) for L in KD]
    
    for Z1,Z2 in zip(z[:-1],z[1:]):
        KK[Z1,Z2]+=1
    KK=[np.array(L)/sum(L) for L in KK]
    return KK, KD

def training_by_counting(K, D, x, z):
    """
    Returns a HMM trained on x and z cf. training-by-counting.
    """
    x = translate_observations_to_indices(x)
    z = translate_z_to_indices(z)
    #z = translate_path_to_indices(z)
    
    trans_probs, emission_probs = count_transitions_and_emissions(K, D, x, z)
    
    initial = np.zeros((K))
    for i in range(K):
        if i == z[0]:
            initial[i] = 1
        else:
            continue
    
    HMM = hmm(initial,trans_probs,emission_probs)
    return HMM

def compute_w_log(model, x):
    k = len(model.init_probs)
    n = len(x)
    
    w = np.zeros((k, n))
    x = translate_observations_to_indices(x)
    
    # Base case: fill out w[i][0] for i = 0..k-1
    # ...
    for i in range(k):
        w[i,0] = log(model.init_probs[i]) + log(model.emission_probs[i][x[0]])
    
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    # ...
    for j in range(1,n):
        for i in range(k):
            w[i,j] = float('-inf')
            for l in range(k):
                w[i,j] = max(w[i,j],log(model.emission_probs[i][x[j]]) + w[l,j-1] + log(model.trans_probs[l][i]))
    return w

def opt_path_prob_log(w):
    p = np.max(w[:,-1])
    return p

def backtrack_log(model, x, w):
    k = len(model.init_probs)
    n = len(x)
    x = translate_observations_to_indices(x)
    z = np.zeros((n))
    z = [int(i) for i in z]
    x = [int(i) for i in x]
    
    lis = w[:,-1]
    lis = lis.tolist()
    z[n-1] = lis.index(np.max(lis))
    
    for i in reversed(range(0,n-1)):
        listen = []
        for j in range(k):
            listen.append(log(model.emission_probs[z[i+1]][x[i+1]]) + w[j,i] + log(model.trans_probs[j][z[i+1]]))
        z[i] = listen.index(np.max(listen))
    return z

def viterbi_update_model(model, x):
    """
    return a new model that corresponds to one round of Viterbi training, 
    i.e. a model where the parameters reflect training by counting on x 
    and z_vit, where z_vit is the Viterbi decoding of x under the given 
    model.
    """
    
    # Your code here ...
    w = compute_w_log(model, x)
    z_vit = backtrack_log(model, x, w)
    emission_probs = np.array(model.emission_probs)
    K = emission_probs.shape[0]
    D = emission_probs.shape[1]
    new_model = training_by_counting(K, D, x, z_vit)
    
    return new_model

#find z for unknown genomes
def compute_annotation(model, obs):
    w = compute_w_log(model, obs)
    z_vit = backtrack_log(model, obs, w)
    z = translate_indices_to_path(z_vit)
    return z

#accuracy (ACC)
def compute_accuracy(true_ann, pred_ann):
    if len(true_ann) != len(pred_ann):
        return 0.0
    return sum(1 if true_ann[i] == pred_ann[i] else 0 
               for i in range(len(true_ann))) / len(true_ann)

In [2]:
def count_c(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c':
            total = total + 1
            if true[i] == 'C' or true[i] == 'c':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'R' or true[i] == 'r':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_r(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if true[i] == 'R' or true[i] == 'r':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'C' or true[i] == 'c':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_cr(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c' or pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if (pred[i] == 'C' or pred[i] == 'c') and (true[i] == 'C' or true[i] == 'c'):
                tp = tp + 1
            elif (pred[i] == 'R' or pred[i] == 'r') and (true[i] == 'R' or true[i] == 'r'):
                tp = tp + 1                
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def safe_div(a, b):
    if b != 0:
        return a/b
    else:
        return float("inf")

def print_stats(tp, fp, tn, fn):

    # sn = float(tp) / (tp + fn) 
    # sp = float(tp) / (tp + fp)
    # acp = 0.25 * (float(tp)/(tp+fn) + float(tp)/(tp+fp) + float(tn)/(tn+fp) + float(tn)/(tn+fn))

    sn = safe_div(float(tp), (tp + fn))
    sp = safe_div(float(tp), (tp + fp))
    acp = 0.25 * (safe_div(float(tp), (tp+fn)) + safe_div(float(tp), (tp+fp)) + safe_div(float(tn), (tn+fp)) + safe_div(float(tn), (tn+fn)))

    ac = (acp - 0.5) * 2
    print("Sn = %.4f, Sp = %.4f, AC = %.4f" % (sn, sp, ac))

def print_all(true, pred):
    (totalc, tp, fp, tn, fn) = count_c(true, pred)
    if totalc > 0:
        print("Cs   (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn), end=" ")
        print_stats(tp, fp, tn, fn)

    (totalr, tp, fp, tn, fn) = count_r(true, pred)
    if totalr > 0:
        print("Rs   (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn), end=" ")
        print_stats(tp, fp, tn, fn)

    (total, tp, fp, tn, fn) = count_cr(true, pred)
    if totalc > 0 and totalr > 0:
        print("Both (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn), end=" ")
        print_stats(tp, fp, tn, fn)

In [3]:
g1 = read_fasta_file('genome1.fa')
g2 = read_fasta_file('genome2.fa')
g3 = read_fasta_file('genome3.fa')
g4 = read_fasta_file('genome4.fa')
g5 = read_fasta_file('genome5.fa')
g6 = read_fasta_file('genome6.fa')
g7 = read_fasta_file('genome7.fa')
g8 = read_fasta_file('genome8.fa')
g9 = read_fasta_file('genome9.fa')
g10 = read_fasta_file('genome10.fa')
z1 = read_fasta_file('true-ann1.fa')
z2 = read_fasta_file('true-ann2.fa')
z3 = read_fasta_file('true-ann3.fa')
z4 = read_fasta_file('true-ann4.fa')
z5 = read_fasta_file('true-ann5.fa')
N = 100
g1 = g1['genome1']#[:N]
g2 = g2['genome2']#[:N]
g3 = g3['genome3']#[:N]
g4 = g4['genome4']#[:N]
g5 = g5['genome5']#[:N]
g6  = g6['genome6']#[:N]
g7  = g7['genome7']#[:N]
g8  = g8['genome8']#[:N]
g9  = g9['genome9']#[:N]
g10 = g10['genome10']#[:N]
z1 = z1['true-ann1']#[:N]
z2 = z2['true-ann2']#[:N]
z3 = z3['true-ann3']#[:N]
z4 = z4['true-ann4']#[:N]
z5 = z5['true-ann5']#[:N]

In [4]:
#print(z1)

In [5]:
A = []
pi = []
phi = []
xdict = {'g1': g1, 'g2': g2, 'g3': g3, 'g4': g4, 'g5': g5}
zdict = {'z1': z1, 'z2': z2, 'z3': z3, 'z4': z4, 'z5': z5}
for i in range(5):
    x = 'g%s'%(i+1)
    z = 'z%s'%(i+1)
    model = training_by_counting(7,4,xdict[x],zdict[z])
    A.append(model.trans_probs)
    pi.append(model.init_probs)
    phi.append(model.emission_probs)

A = np.array(A)
pi = np.array(pi)
phi = np.array(phi)

In [6]:
Aave = np.sum(A,axis=0)/4
piave = np.sum(pi,axis=0)/4
phiave = np.sum(phi,axis=0)/4
xdict = {'g1': g1, 'g2': g2, 'g3': g3, 'g4': g4, 'g5': g5}
zdict = {'z1': z1, 'z2': z2, 'z3': z3, 'z4': z4, 'z5': z5}
for i in range(5):
    Ai = Aave-A[i]/4
    pii = piave-pi[i]/4
    phii = phiave-phi[i]/4
    x = 'g%s'%(i+1)
    z = 'z%s'%(i+1)
    ztrue = zdict[z]
    HMM = hmm(pii,Ai,phii)
    zann = compute_annotation(HMM, xdict[x])
    #print(z)
    #print(zann)
    if len(ztrue) != len(zann):
        print("ERROR: The lengths of two predictions are different")
        print("Expected %d, but found %d" % (len(true_ann), len(pred_ann)))
        sys.exit(1)
    print_all(ztrue, zann)

    #print(i)

Cs   (tp=637950, fp=162347, tn=310617, fn=85600): Sn = 0.8817, Sp = 0.7971, AC = 0.5598
Rs   (tp=536339, fp=119588, tn=324794, fn=71423): Sn = 0.8825, Sp = 0.8177, AC = 0.6254
Both (tp=1174289, fp=281935, tn=239194, fn=157023): Sn = 0.8821, Sp = 0.8064, AC = 0.3756
Cs   (tp=726234, fp=153460, tn=324779, fn=91488): Sn = 0.8881, Sp = 0.8256, AC = 0.5865
Rs   (tp=771727, fp=143797, tn=325437, fn=90830): Sn = 0.8947, Sp = 0.8429, AC = 0.6065
Both (tp=1497961, fp=297257, tn=233949, fn=182318): Sn = 0.8915, Sp = 0.8344, AC = 0.3642
Cs   (tp=678179, fp=174832, tn=526957, fn=116039): Sn = 0.8539, Sp = 0.7950, AC = 0.6097
Rs   (tp=830905, fp=172367, tn=506243, fn=136753): Sn = 0.8587, Sp = 0.8282, AC = 0.6101
Both (tp=1509084, fp=347199, tn=390204, fn=252792): Sn = 0.8565, Sp = 0.8130, AC = 0.4027
Cs   (tp=569659, fp=133317, tn=338354, fn=92348): Sn = 0.8605, Sp = 0.8104, AC = 0.5869
Rs   (tp=538399, fp=124769, tn=334589, fn=96113): Sn = 0.8485, Sp = 0.8119, AC = 0.5828
Both (tp=1108058, fp=258

In [18]:
#print(Aave)

def pretty(A,epsilon):
    res = np.zeros(A.shape)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i][j] > epsilon:
                res[i][j] = A[i][j]
            
    return res

print(pretty(Aave,0.01))
print(Aave)

[[0.         0.         1.24568211 0.         0.         0.
  0.        ]
 [1.24997214 0.         0.         0.         0.         0.
  0.        ]
 [0.         1.24997214 0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         1.24581714 0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         1.24997143
  0.        ]
 [0.         0.         0.         0.         0.         0.
  1.24997143]
 [0.         0.         0.         0.         1.24579217 0.
  0.        ]]
[[4.64351339e-06 4.64351339e-06 1.24568211e+00 4.29242655e-03
  6.89220090e-06 4.64351339e-06 4.64351339e-06]
 [1.24997214e+00 4.64351339e-06 4.64351339e-06 4.64351339e-06
  4.64351339e-06 4.64351339e-06 4.64351339e-06]
 [4.64351339e-06 1.24997214e+00 4.64351339e-06 4.64351339e-06
  4.64351339e-06 4.64351339e-06 4.64351339e-06]
 [2.28375194e-06 2.28375194e-06 2.13474915e-03 1.24581714e+00
  2.03898027e-03 2.28375194e-06 2.28375194e-06]
 [4.76230589e-06 4.76230589e-06 4

In [7]:
print(g1)

...


In [8]:
"""new_fasta="1234123"
with open('test.fa', 'w') as f:
    f.write("".join(";NC_007622.1 Staphylococcus aureus RF122, complete genome.\n>genome%d\n" % 7))
    #f.write("\n".join(" "))
    f.write(''.join(g1[:1000]))
test = read_fasta_file("test.fa")
print(test)"""

'new_fasta="1234123"\nwith open(\'test.fa\', \'w\') as f:\n    f.write("".join(";NC_007622.1 Staphylococcus aureus RF122, complete genome.\n>genome%d\n" % 7))\n    #f.write("\n".join(" "))\n    f.write(\'\'.join(g1[:1000]))\ntest = read_fasta_file("test.fa")\nprint(test)'

In [9]:
Aave5 = np.sum(A,axis=0)/5
piave5 = np.sum(pi,axis=0)/5
phiave5 = np.sum(phi,axis=0)/5
HMM = hmm(piave5,Aave5,phiave5)
z6true = compute_annotation(HMM,g6)
z7true = compute_annotation(HMM,g7)
z8true = compute_annotation(HMM,g8)
z9true = compute_annotation(HMM,g9)
z10true = compute_annotation(HMM,g10)

In [10]:
with open('pred-ann6.fa', 'w') as f:
    f.write('>pred-ann6 \n')
    f.write(''.join(z6true))
    f.write('\n')
with open('pred-ann7.fa', 'w') as f:
    f.write('>pred-ann7 \n')
    f.write(''.join(z7true))
    f.write('\n')
with open('pred-ann8.fa', 'w') as f:
    f.write('>pred-ann8 \n')
    f.write(''.join(z8true))
    f.write('\n')
with open('pred-ann9.fa', 'w') as f:
    f.write('>pred-ann9 \n')
    f.write(''.join(z9true))
    f.write('\n')
with open('pred-ann10.fa', 'w') as f:
    f.write('>pred-ann10 \n')
    f.write(''.join(z10true))
    f.write('\n')

In [11]:
with open('pred-ann.fa', 'w') as f:
    f.write('>pred-ann6 \n')
    f.write(''.join(z6true))
    f.write('\n')
    f.write('>pred-ann7 \n')
    f.write(''.join(z7true))
    f.write('\n')
    f.write('>pred-ann8 \n')
    f.write(''.join(z8true))
    f.write('\n')
    f.write('>pred-ann9 \n')
    f.write(''.join(z9true))
    f.write('\n')
    f.write('>pred-ann10 \n')
    f.write(''.join(z10true))
    f.write('\n')

In [12]:
np.sqrt(2)

1.4142135623730951